In [1]:
import os, shutil
original_dataset_dir = "cats_vs_dogs/train"
dirs = []
base_dir = "cats_vs_dogs_small"
dirs.append(base_dir)
train_dir = os.path.join(base_dir, "train")
dirs.append(train_dir)
validation_dir = os.path.join(base_dir, "validation")
dirs.append(validation_dir)
test_dir = os.path.join(base_dir, "test")
dirs.append(test_dir)
train_cats_dir = os.path.join(train_dir, "cats")
dirs.append(train_cats_dir)
train_dogs_dir = os.path.join(train_dir, "dogs")
dirs.append(train_dogs_dir)
validation_cats_dir = os.path.join(validation_dir, "cats")
dirs.append(validation_cats_dir)
validation_dogs_dir = os.path.join(validation_dir, "dogs")
dirs.append(validation_dogs_dir)
test_cats_dir = os.path.join(test_dir, "cats")
dirs.append(test_cats_dir)
test_dogs_dir = os.path.join(test_dir, "dogs")
dirs.append(test_dogs_dir)
for directory in dirs:
    if not os.path.exists(directory):
       os.mkdir(directory)
    
fnames = ["cat.{}.jpg".format(i) for i in range(1000)]
for fname in fnames:
    src = os.path.join(original_dataset_dir, fname)
    dest = os.path.join(train_cats_dir, fname)
    shutil.copy(src, dest)
fnames = ["cat.{}.jpg".format(i) for i in range(1000, 1500)]
for fname in fnames:
    src = os.path.join(original_dataset_dir, fname)
    dest = os.path.join(validation_cats_dir, fname)
    shutil.copy(src, dest)
fnames = ["cat.{}.jpg".format(i) for i in range(1500, 2000)]
for fname in fnames:
    src = os.path.join(original_dataset_dir, fname)
    dest = os.path.join(test_cats_dir, fname)
    shutil.copy(src, dest)
   
fnames = ["dog.{}.jpg".format(i) for i in range(1000)]
for fname in fnames:
    src = os.path.join(original_dataset_dir, fname)
    dest = os.path.join(train_dogs_dir, fname)
    shutil.copy(src, dest)
fnames = ["dog.{}.jpg".format(i) for i in range(1000, 1500)]
for fname in fnames:
    src = os.path.join(original_dataset_dir, fname)
    dest = os.path.join(validation_dogs_dir, fname)
    shutil.copy(src, dest)
fnames = ["dog.{}.jpg".format(i) for i in range(1500, 2000)]
for fname in fnames:
    src = os.path.join(original_dataset_dir, fname)
    dest = os.path.join(test_dogs_dir, fname)
    shutil.copy(src, dest)
for directory in dirs:
    print(directory, ":", len(os.listdir(directory)))
    
print("Done.")

cats_vs_dogs_small : 3
cats_vs_dogs_small/train : 2
cats_vs_dogs_small/validation : 2
cats_vs_dogs_small/test : 2
cats_vs_dogs_small/train/cats : 1000
cats_vs_dogs_small/train/dogs : 1000
cats_vs_dogs_small/validation/cats : 500
cats_vs_dogs_small/validation/dogs : 500
cats_vs_dogs_small/test/cats : 500
cats_vs_dogs_small/test/dogs : 500
Done.


In [3]:
from keras import models
from keras import layers

model = models.Sequential()
# Feature extraction
model.add(layers.Conv2D(32, (3, 3),
                        activation="relu", 
                        input_shape=(150, 150, 3)))# Feature detection, 3 colors
model.add(layers.MaxPooling2D((2,2))) # Downsampling

# number of features = 64
model.add(layers.Conv2D(64, (3, 3),
                        activation="relu")) # Feature detection
model.add(layers.MaxPooling2D((2,2)))  # Downsampling

# look for big areas
model.add(layers.Conv2D(128, (3, 3),
                        activation="relu")) # Feature detection
model.add(layers.MaxPooling2D((2,2)))  # Downsampling

model.add(layers.Conv2D(128, (3, 3),
                        activation="relu")) # Feature detection
model.add(layers.MaxPooling2D((2,2)))  # Downsampling

# Embedding, vector store and embeddeing the content of the features
model.add(layers.Flatten())
# Classifier
model.add(layers.Dense(512, activation="relu"))
model.add(layers.Dense(1, activation="sigmoid"))
model.summary() 

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 148, 148, 32)      896       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 74, 74, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 72, 72, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 36, 36, 64)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 34, 34, 128)       73856     
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 17, 17, 128)       0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 15, 15, 128)       147584    
__________

In [4]:
from keras import optimizers
model.compile(
    optimizer = optimizers.RMSprop(lr=1e-4),
    loss = "binary_crossentropy",
    metrics = ["accuracy"]
)

In [ ]:
from keras.preprocessing.image import ImageDataGenerator
# rescale the colors
'''
Data augmentation
put this
datagen = ImageDataGenerator(
    rotation_range = 40,
    width_shift_range = 0.2,
    height_shift_range = 0.2,
    shear_range = 0.2,
    zoom_range = 0.2,
    horizontal_flip = True,
    fill_mode = "nearest"    
)
instead of this ImageDataGenerator
'''

train_datagen = ImageDataGenerator(rescale=1./255.)
validation_datagen = ImageDataGenerator(rescale=1./255.)

#input,output image and label
train_datagenerator = train_datagen.flow_from_directory(
    train_dir,
    target_size = (150,150),
    batch_size = 20,
    class_mode = "binary"
)

validation_generator = validation_datagen.flow_from_directory(
    validation_dir,
    target_size = (150,150),
    batch_size = 20,
    class_mode = "binary"
)

In [8]:
def generator():
    i = 0
    while True:
        i += 1
        yield i
        


<generator object generator at 0x7f825d109830>

In [11]:
# we don't need to provide labels because the generator will provide the labels
history = model.fit_generator(
    train_datagenerator,
    steps_per_epoch = 30,
    epochs=30,
    validation_data = validation_generator,
    validation_steps = 50
)

Epoch 1/30
30/30 [==============================] - 60s 2s/step - loss: 0.7020 - acc: 0.4900 - val_loss: 0.6877 - val_acc: 0.5010